<a href="https://colab.research.google.com/github/loosak/pysnippets/blob/master/01_time_series_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Optional: change Jupyter Notebook theme to GDD theme
from IPython.core.display import HTML
HTML(url='https://gdd.li/jupyter-theme')


![footer_logo](https://github.com/JTHaywardGDD/time-series/blob/main/images/logo.png?raw=1)
# Time Series Analysis

- [repo:](https://github.com/JTHaywardGDD/time-series)
- [video](https://youtu.be/2b725bplNt8)

## Goal

Pandas is the core data manipulation and analysis library for Python and it has some amazing utilities for dealing with series time data. 

The goal of this notebook is to familiarise ourselves with how Pandas can be used to work with Time Series data. 

We shall use a real Time Series dataset to demonstrate these functionalities and learn some fundamental techniques for Time Series analysis.

## Program
1. [Time Utilities in Pandas](#timeutil)
2. [Reading in Time Series Data](#read)
3. [Time-based Manipulations](#mani)
4. [Smoothing](#roll)
5. [Summary](#sum)


In [2]:
import pandas as pd

<a id='timeutil'></a>

## 1. Time Utilities in Pandas

### Timestamps
![footer_logo](https://github.com/JTHaywardGDD/time-series/blob/main/images/clock.jpeg?raw=1)

In pandas, specific times are represented as **Timestamps**. A Timestamp is the pandas equivalent of python’s Datetime and is interchangeable with it in most cases.

Pandas can create datetime data from strings formated as `'yyyy-mm-ddThh:mm:ss:ms'` using `pd.Timestamp()`. The date units are years (‘Y’), months (‘M’), weeks (‘W’), and days (‘D’), while the time units are hours (‘h’) in 24 hour format, minutes (‘m’), seconds (‘s’), milliseconds (‘ms’). Note that time units are combined with date units using `'T'

In [3]:
date = pd.Timestamp('2022-03-26T09:00:00')
print(f'the date is {date.date()} and the time is {date.time()}')
date

the date is 2022-03-26 and the time is 09:00:00


Timestamp('2022-03-26 09:00:00')

Pandas Timestamps support a wide range of [operations](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Timestamp.html).

For example, we can access various attributes stored in the Timestamp.

In [4]:
pd.Timestamp('2022-02-20T18:34:56').daysinmonth

28

In [5]:
pd.Timestamp('2022-02-20T18:34:56').weekofyear

7

In [6]:
pd.Timestamp('2022-02-20T18:34:56').quarter

1

We can also perform time based operations and use time related methods.

In [ ]:
pd.Timestamp('2022-02-20T18:34:56') - pd.Timestamp('2020-02-18T18:24:32')

In [ ]:
pd.Timestamp('2022-02-20T18:34:56').month_name()

In [ ]:
pd.Timestamp('2022-02-20T18:34:56').day_name()

<a id='ex'></a>
### <mark>Exercise: Investigate the timestamp features and methods

We've seen a few examples, but let's investigate further.
- What day of the year is it today
- Are we in a leap year?
- How long is it until a Public Holiday (e.g. Christmas)?


In [7]:
current_date = pd.Timestamp('today')
current_date

Timestamp('2022-05-24 18:41:05.746409')

<a id='read'></a>
![footer_logo](https://github.com/JTHaywardGDD/time-series/blob/main/images/air-quality.jpeg?raw=1) 


## 2. Reading in Time Series Data


Throughout this taster will use a dataset containing daily air quality index in Californian counties between 2007 and 2017 (based on a larger dataset from [Kaggle](https://www.kaggle.com/epa/carbon-monoxide)). 

Each datapoint indicates the average air quality index on a certain day: the higher the value - the more polluted.

In [8]:
air_df = pd.read_csv('data/air_quality.csv')
air_df.head()

FileNotFoundError: ignored

Typically time data is contained in a separate column of standard strings; notice how our time data is not currently recognised as Timestamps.

In [ ]:
air_df.info()

In order to make our time data machine readable, we can set `parse_dates` with the list of columns to be converted to Pandas Timestamps when reading the data with `pd.read_csv`. This automatically identifies the format of the dates, although specific formatting is also possible.

For most time-series analysis functionality, we also benefit from setting the dates as the index in the Pandas DataFrame. 

In [ ]:
air_df = pd.read_csv('data/air_quality.csv', index_col='date_local', parse_dates=True)
air_df.head()

In [ ]:
air_df.info()

With the Timestamps as the index, we can directly filter on our DataFrame using the`loc` method and easily produce plots to visualise our data.

In [ ]:
air_df.loc['2008-02-01':'2008-02-10']

In [ ]:
air_df.plot(figsize=(16,4));

<a id='mani'></a>

## 3. Time-based manipulations

### Easy Aggregations

Another advantage of the datetime-index approach is that it provides us with some functionality for easy time-based aggregations. One such aggregation is `resample`. 

For example, we can easily calculate the _mean_ per _year_ by running:

In [ ]:
air_df.resample('Y').mean()

You can also run the same aggregation per month `M`, week `W`, day `D` or quarter `Q`. Custom aggregation periods are also possible, for example per 4 weeks `4W` or per 3 months `3M`. If the index is a timestamp that also includes times, then you can also aggregate per hour. See [here] for a more comprehensive list of offsets, that can be as specific as _'Business Month Begin'_. 

[here]: https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#dateoffset-objects

In the cell below we compute the mean AQI for consecutive 4 week periods.

In [ ]:
air_df.resample('4W').mean().head()

We can also use general `.agg()` methods here to apply multiple aggregators, including custom aggregations. For example, the spread per month: 

In [ ]:
(
    air_df
    .resample('M')
    .agg([
        ('mean','mean'),
        ('var','var'),
        ('spread', lambda month_df: month_df.max() - month_df.min())
    ])
    #.droplevel(0, axis=1)
    .head()
)

### Time Based Features 

Any column in Pandas that is of dtype `datetime` has a module attached that can be used to perform vectorised datetime operations. This is very similar to the `.str` module attached to string columns. It is a good thing to explore since the alternative is non-vectorised and much slower.

Below is an example of getting the quarter and adding the day name. Feel free to explore [other](https://pandas.pydata.org/pandas-docs/stable/reference/series.html#api-series-dt) properties and methods.

In [ ]:
(
    air_df
    .assign(quarter = lambda df: df.index.quarter,
            weekday = lambda df: df.index.day_name())
    .head()
)

### <mark>Exercise: Resampling

Compute the average AQI per week and find the week that had the worst (or highest) AQI




In [ ]:
# %load answers/resample.py

### <mark>Exercise: Weekday with worst air quality</mark>

Which day of the week has the worst (or highest) on average? Does AQI drop in the weekend? 

 <font color='green'>Bonus </font>: Make a bar plot to better illustrate any weekly air quality patterns

In [ ]:
# %load answers/time_based_features.py

### <font color='green'>BONUS SECTION: </font> Shifting

It can be often useful to shift some variables forward or backwards in time. This can for example help us create variables with lagged values or calculate differences in values between time steps. This can be done using Panda's `shift` method, which can shift values by a given number of periods (positive or negative). 

The example below uses this method to create a new variable for AQI during the previous day:

In [ ]:
(
    air_df
    .assign(aqi_yesterday = lambda df: df.aqi.shift(1))
    .assign(change_in_aqi = lambda df: df.aqi - df.aqi_yesterday)
    .dropna()
    .head()
)



<a id='roll'></a>

## 4. Smoothing

Let us have a closer look at the air quality patterns during a single year. 

The simplest way to plot timestamp data dynamics in Pandas is using the `plot` method, which by default plots a linear plot over time:

In [ ]:
air_df_2016 = air_df.loc['2016']

air_df_2016.plot(figsize=(18,6));

### Rolling Average smoothing

Various patterns can be seen on this daily line graph, but the overall trend can be hard to see due to noise plentiful short spikes. 

In order to see a *smoother* pattern over time, a __rolling average__ can be applied to a Time Series. It walks over the timestamps with a given window (7 days for example) and calculates averages for each. 

Pandas can perform this via the `rolling` method which can be called on both a DataFrame as well as a Series object. The window size for this method can be set using both a fixed number of data points as well as particular time intervals (days, weeks etc).

In [ ]:
(
    air_df_2016
    .assign(rolling_mean=lambda df: df['aqi'].rolling('20D').mean())
    .plot(figsize=(16, 4))
);

Note that the orange *rolling mean line* is lagging behind what actually happens. This is because, by default, each point of the rolling average represents information about this day and the preceding days - not just this particular moment! We can remove this effect using *centering*.

To center the rolling mean, we can either manually shift it backwards or use the option `center=True` for the `rolling()` method. You can see below how both achieve the same result - the red and green (overlapping) lines do not lag anymore. 

__Importantly__, centering requires information from the future for each point. This makes centering bad practice if we want to further make predictions about the future — this information is then already contained in the present data points, which is referred to as *information leakage* in machine learning.

In [ ]:
(
    air_df_2016
    .assign(
        rolling_mean=lambda df: df['aqi'].rolling('20D').mean(),
        # With `center=True` window size cannot be a time frame!
        rolling_mean_center=lambda df: df['aqi'].rolling(20, center=True).mean(),
        manual_center=lambda df: df['rolling_mean'].shift(-9)
    )
    .plot(figsize=(16,4), title='rolling_mean_center and manual_center overlap')
);

Rolling average smoothing is a simple way to isolate signal from noise in Time Series data and get an idea about general Time Series behavior.

However, there are some notable drawbacks:

- Highly dependent on window size: 
    - using a small window sizes can lead to more noise than signal;
    - using a large window size can remove important signal information).
- It always lags by the window size (unless centered).
- It is not really informative about the future.
- It can be significantly skewed by extreme datapoints in the past.

### <mark>Exercise: Weekday with worst air quality</mark>

In a previous exercise we saw that the week commencing Monday 24th December 2007 had the worst average air quality.


In [ ]:
(
    air_df
    .resample('W-Mon')
    .mean()
    .nlargest(1, 'aqi')
)

However, this is not necessarilly the 7-day period that had the worst average air quality.

For example, that may span a Friday to Thursday, as opposed to Monday to Sunday.

Find the 7-day period that had the worst average air quality.


In [ ]:
# %load answers/rolling.py

### <font color='green'>BONUS SECTION: </font> Exponential Smoothing

An alternative to calculating the rolling statistics is to smooth the timeseries exponentially with the following formula:

$$\hat{y_t} = \alpha y_t + (1-\alpha) \hat{y}_{t-1}$$

where $\hat{y_t}$ is the output of the exponential smoothing at time $t$, $y_t$ is the data point at time $t$, and $0<\alpha<1$ is the *the smoothing factor*.


The idea is to recursively smooth the series by averaging the current average with the current value. If $\alpha$ is high then the smoothing will be low but the average can respond quicker to changes, and if it is low — the result will be much more smooth and flat.

In [ ]:
(
    air_df_2016
    .assign(
        smoothed_01=lambda df: df['aqi'].ewm(alpha=0.1).mean(),
        smoothed_001=lambda df: df['aqi'].ewm(alpha=0.01).mean()
    )
    .plot(figsize=(16, 4))
);

Exponential Smoothing exhibits reduced lagging and more weight assigned to the current timestamps compared to Simple Rolling Averages, which also makes it more informative about the future. You can read more about EWM (exponentially weighted Moving average) in the pandas [docs].

[docs]: https://pandas.pydata.org/pandas-docs/stable/user_guide/computation.html#exponentially-weighted-windows

Other alternatives are [weighted smoothing](http://pandas.pydata.org/pandas-docs/stable/user_guide/computation.html#rolling-windows) and [expanding windows](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.expanding.html). 

<a id='sum'></a>
## 5. Summary

We have covered: 
- Timestamps and formatting in Pandas
- How to properly read in Time Series data in Pandas, and why it is important to set the date as an index
- Time based manipulations, such as aggregations with `resample`, time-based features based on dtype `datetime` and shifting
- Smoothing with rolling averages, its disadvantages and some alternatives.

We should now be able to answer analytics questions like: 
- Which year had the worst air quality?
- Which five day period had the largest decrease in air quality (between the first and last day)?
- Does air quality improve over the weekend? 
